In [1]:
import pandas as pd
import numpy as np
import os
import glob
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd
import wfdb
import statsmodels.api as sm
import matplotlib.image as mpimg
import tkinter as tk
from RDFunctions import *

In [2]:
df = pd.read_csv('RecordsAnno4.csv',sep=',')
dfx = df[['RecordNum','PatientId','ICUStayId','Duration','Category']]
dfx.loc[:,'nRRraw'] = 0 # number of raw RR alarms in full duration
dfx.loc[:,'nRRfilt'] = 0 # number of filtered RR alarms in full duration
dfx.loc[:,'nSpO2raw'] = 0 # number of raw SpO2 alarms in full duration
dfx.loc[:,'nSpO2filt'] = 0 # number of filtered SpO2 alarms in full duration
dfx.loc[:,'nRD'] = 0 # number of RD alarms obviously occuring after first 24 hours
dfx.loc[:,'fRR_b24'] = None # first RR alarm occurence before 24 hours, obviously filtered RR alarm
dfx.loc[:,'fRR_a24'] = None # first RR alarm occurence after 24 hours, obviously filtered RR alarm
dfx.loc[:,'fSP_b24'] = None # first SpO2 alarm occurence before 24 hours, obviously filtered SpO2 alarm
dfx.loc[:,'fSP_a24'] = None # first SpO2 alarm occurence after 24 hours, obviously filtered SpO2 alarm
dfx.loc[:,'fRD'] = None # first RD alarm, obviously after 24 hours
dfx.loc[:,'Lead'] = None # Lead of RD alarm with respect of filtered alarm
dfx.head()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,RecordNum,PatientId,ICUStayId,Duration,Category,nRRraw,nRRfilt,nSpO2raw,nSpO2filt,nRD,fRR_b24,fRR_a24,fSP_b24,fSP_a24,fRD,Lead
0,3887555,79,290076,38.200000,D,0,0,0,0,0,None,None,None,None,None,None
1,3647298,85,291697,45.316667,B,0,0,0,0,0,None,None,None,None,None,None
2,3485814,123,227264,34.000000,C,0,0,0,0,0,None,None,None,None,None,None
3,3991520,124,255660,53.016667,F,0,0,0,0,0,None,None,None,None,None,None
4,3807277,124,255660,32.316667,F,0,0,0,0,0,None,None,None,None,None,None


In [3]:
OUTLIERSD = 2.5
LOWESSWINDOW = 3.0
THRESHOLD_RR = 24.0
THRESHOLD_SPO2 = 92.0

for i in range(len(df)):
    print(i,end=" ")
    record_name = dfx.loc[i,'RecordNum']
    flname = './FinalRecords/'+str(record_name)+'n.hea'
    recname = './FinalRecords/'+str(record_name)+'n'      
    [samples,R,S,H,firstline] = getIndexOfRRSpO2HR(flname)
    rec =  wfdb.io.rdrecord(str(recname))
    xrr = rec.p_signal[:,R]
    xspo2 = rec.p_signal[:,S]
    xhr = rec.p_signal[:,H]
    TOTAL_LEN = len(xrr)
    t = np.arange(0,TOTAL_LEN,1)
    
    [xrrnew,trrnew] = outlierRejector(xrr,t,OUTLIERSD,default=15.0)
    [xspo2new,tspo2new] = outlierRejector(xspo2,t,OUTLIERSD,default=98.0)
    [xhrnew,thrnew] = outlierRejector(xhr,t,OUTLIERSD)
    
    zrrnew = applyLowess(xrrnew,trrnew,LOWESSWINDOW*60)
    zspo2new = applyLowess(xspo2new,tspo2new,LOWESSWINDOW*60)
    zhrnew = applyLowess(xhrnew,thrnew,LOWESSWINDOW*60)
    
    t = trrnew/60.0
    tnew = zrrnew[:,0]/60.0
    
    filtSpO2Alarms = getFilteredLowAlarm(zspo2new[:,1],tnew,THRESHOLD_SPO2)
    filtRRAlarms = getFilteredHighAlarm(zrrnew[:,1],tnew,THRESHOLD_RR)
    
    SP_b24 = [v for v in filtSpO2Alarms if v<24.0]
    SP_a24 = [v for v in filtSpO2Alarms if v>=24.0]
    
    RR_b24 = [v for v in filtRRAlarms if v<24.0]
    RR_a24 = [v for v in filtRRAlarms if v>=24.0]
    
    rawSpO2Alarms = getRawLowAlarm(xspo2new,t,THRESHOLD_SPO2)
    rawRRAlarms = getRawHighAlarm(xrrnew,t,THRESHOLD_RR)
    
    dfx.loc[i,'nRRraw'] = len(rawRRAlarms)
    dfx.loc[i,'nRRfilt'] = len(filtRRAlarms)
    dfx.loc[i,'nSpO2raw'] = len(rawSpO2Alarms)
    dfx.loc[i,'nSpO2filt'] = len(filtSpO2Alarms)
    
    if(len(SP_b24)>0):
         dfx.loc[i,'fSP_b24'] = min(SP_b24)
    if(len(SP_a24)>0):
         dfx.loc[i,'fSP_a24'] = min(SP_a24)
            
    if(len(RR_b24)>0):
         dfx.loc[i,'fRR_b24'] = min(RR_b24)
    if(len(RR_a24)>0):
         dfx.loc[i,'fRR_a24'] = min(RR_a24)

dfx.to_csv('AlarmsInfo.csv',index=False)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [4]:
display(dfx)

,RecordNum,PatientId,ICUStayId,Duration,Category,nRRraw,nRRfilt,nSpO2raw,nSpO2filt,nRD,fRR_b24,fRR_a24,fSP_b24,fSP_a24,fRD,Lead
0,3887555,79,290076,38.200000,D,12,0,23,1,0,None,None,11.4333,None,None,None
1,3647298,85,291697,45.316667,B,191,1,8,0,0,1,None,None,None,None,None
2,3485814,123,227264,34.000000,C,0,0,0,0,0,None,None,None,None,None,None
3,3991520,124,255660,53.016667,F,0,0,1,0,0,None,None,None,None,None,None
4,3807277,124,255660,32.316667,F,0,0,0,0,0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,3912588,98481,290141,94.383333,X,0,0,0,0,0,None,None,None,None,None,None
908,3768185,98481,290141,47.600000,A,0,0,48,1,0,None,None,None,30.45,None,None
909,3391887,98481,290141,94.416667,A,153,3,101,1,0,None,24.6833,1,None,None,None
910,3831114,99783,236983,68.566667,B,309,1,0,0,0,15.9,None,None,None,None,None
